# How to write data to an H5 files

Here we are going to utilize a Python library called H5py to write our data to a file.  

We will then load the file back in and analyze the structure

First we will define the libraries that we need.

In [ ]:
import os
import sys
import h5py
import pandas as pd
import numpy as np

#The things below are just needed to open analyze the structure.  No need to import these if you're just
#writing out the h5 file.
path_to_src = os.environ.get('DEV_DIR','')
if not path_to_src:
    print('DEV_DIR environment Variable needs to be set!!!')
sys.path.append(path_to_src)
sys.path.pop(0)
import utils.PhobosFunctions as PF
from utils.fileUtils import genDSDict
try:
    from ipytree import Tree, Node
    GTREE = True
except:
    GTREE = False

We will create a series of Pandas DataFrames to store into an h5 file.  This step would be replaced by whatever data that you want to store.

I'm just going to make a list of DataFrames to start with.

In [ ]:
dflist = []
for i in range(4):
    y = pd.DataFrame()
    y['Time'] = np.linspace(0,1000,500)
    y['Altitude'] = -.5*9.8*y['Time']**2
    dflist.append(y)
dflist

Now we have our data, and are ready to write the data to an H5 file. 

The way data is stored inside an h5 file is through a series of groups and datasets.  The following diagram will show a basic outline of how H5 stores it's data.  

NOTE:  This has no bearing on writing data to H5 files... it is just a visual aid.

In [ ]:
if GTREE:
    tree = Tree(stripes=True)
    fnode = Node('MyFile.h5')
    tree.add_node(fnode)

    group1 = Node('Top Level Group 1')
    group2 = Node('Top Level Group 2')
    DataSet1 = Node('Dataset 1')
    DataSet2 = Node('Dataset 2')
    DataSet3 = Node('Dataset 3')
    DataSet4 = Node('Dataset 4')
    header11 = Node('Time')
    header12 = Node('Altitude')
    series11 = Node('Pandas Series[Time]')
    series12 = Node('Pandas Series[Altitude]')

    header21 = Node('Time')
    header22 = Node('Altitude')
    series21 = Node('Pandas Series[Time]')
    series22 = Node('Pandas Series[Altitude]')

    header31 = Node('Time')
    header32 = Node('Altitude')
    series31 = Node('Pandas Series[Time]')
    series32 = Node('Pandas Series[Altitude]')

    header41 = Node('Time')
    header42 = Node('Altitude')
    series41 = Node('Pandas Series[Time]')
    series42 = Node('Pandas Series[Altitude]')

    fnode.add_node(group1)
    fnode.add_node(group2)
    group1.add_node(DataSet1)
    group1.add_node(DataSet2)
    group2.add_node(DataSet3)
    group2.add_node(DataSet4)

    DataSet1.add_node(header11)
    DataSet1.add_node(header12)
    header11.add_node(series11)
    header12.add_node(series12)

    DataSet2.add_node(header21)
    DataSet2.add_node(header22)
    header21.add_node(series21)
    header22.add_node(series22)

    DataSet3.add_node(header31)
    DataSet3.add_node(header32)
    header31.add_node(series31)
    header32.add_node(series32)

    DataSet4.add_node(header41)
    DataSet4.add_node(header42)
    header41.add_node(series41)
    header42.add_node(series42)

    display(tree)

Technically speaking, H5 files can have an arbitrary number of groups and subgroups, however, to be compliant with Ares, NO H5 SHOULD HAVE MORE THAN 1 LAYER OF GROUPS.  So the Tree above is about has hierarchical as the file should ever be.

NOTE:  You can put as many datasets under the groups as you want.  Here we just have 2 under each group.

Now lets start writing the file.
We will emulate the structure above.

In [ ]:
OutDirectory = os.path.join(os.path.expanduser('~'),'Desktop')
with h5py.File(os.path.join(OutDirectory,'MyFile.h5'),'w') as hf:
    j = 0
    for i in range(len(dflist)):
        #Only create a new group when i is 0 or 2, because we want 2 datasets per group... This can obviously
        # be changed in your code, however you want to set up the data.
        if i%2 == 0:
            j+=1
            group = hf.create_group(f'Top Level Group {j}')
        dset = group.create_group(f'Dataset {i+1}')
        #This is where the data is getting written to the h5 file.  It needs to be under a group!
        for key in dflist[i]:
            dset.create_dataset(key,data=dflist[i][key])
genDSDict(os.path.join(OutDirectory,'MyFile.h5'))
        
    

Now to load our data and see if we can see the structure that we have created, utilizing Phobos Functions

In [ ]:
fpath = os.path.join(OutDirectory,'MyFile.h5')
groups = PF.get_groups(fpath)
print(f'MyFile.h5 contains the following groups {groups}\n\n')
for group in groups:
    dsets = PF.get_dsets(fpath,group)
    print(f'\t{group} contains the following datasets {dsets}')
    for dset in dsets:
        headers = PF.get_headers(fpath,group,dset)
        print(f'\t\t{dset} contains the following headers {headers}')
    print('\n')